<a href="https://colab.research.google.com/github/ConanOReilly/Final_Year_Project/blob/main/Image_Data/ModelComparison_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Image Classification Model Comparison**
State-of-art, out-of-box CNNs and ViTs will be trained, evaluated and compared on the PAD-UFES-2020 dataset. The best performing model will be optimised and fine tuned.

Models:
*   DenseNet121
*   ResNet50V2
*   EfficientNetV2B0
*   Swin-Tiny
*   DeiT-Small


# **Papers**


*   Aljohani, Khalil, and Turki Turki. "Automatic classification of melanoma skin cancer with deep convolutional neural networks." Ai 3.2 (2022): 512-525.
*   Kreouzi M, Theodorakis N, Feretzakis G, Paxinou E, Sakagianni A, Kalles D, Anastasiou A, Verykios VS, Nikolaou M. Deep Learning for Melanoma Detection: A Deep Learning Approach to Differentiating Malignant Melanoma from Benign Melanocytic Nevi. Cancers (Basel). 2024 Dec 25;17(1):28. doi: 10.3390/cancers17010028. PMID: 39796659; PMCID: PMC11718884.
*   Ozdemir, B., Pacal, I. A robust deep learning framework for multiclass skin cancer classification. Sci Rep 15, 4938 (2025).
*   Shehzad, K.; Zhenhua, T.; Shoukat, S.; Saeed, A.; Ahmad, I.; Sarwar Bhatti, S.; Chelloug, S.A. A Deep-Ensemble-Learning-Based Approach for Skin Cancer Diagnosis. Electronics 2023, 12, 1342.





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Libraries**


In [ ]:
import os
import torch
import timm
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image
from sklearn.preprocessing import LabelEncoder

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth set successfully.")
    except RuntimeError as e:
        print("RuntimeError:", e)

GPU memory growth set successfully.


# **Preporcessing and Augmentation**

In [ ]:
# Paths
image_dir = "/content/drive/MyDrive/Final Year Project/Data/PAD/imgs"
metadata_path = "/content/drive/MyDrive/Final Year Project/Data/PAD/Metadata/metadata.csv"

# Load metadata
metadata_df = pd.read_csv(metadata_path)

# Encode class labels
label_encoder = LabelEncoder()
metadata_df["label_encoded"] = label_encoder.fit_transform(metadata_df["diagnostic"].str.upper())
class_names = label_encoder.classes_
num_classes = len(class_names)
print("Classes:", class_names)

# Image size
img_size = 224

# Unified transforms for training and validation
unified_train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

unified_val_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Custom Dataset
class SkinCancerDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row["img_id"])

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image: {img_path} | {e}")
            image = Image.new("RGB", (224, 224))

        label = row["label_encoded"]

        if self.transform:
            image = self.transform(image)

        return image, label

# Split into train/val
train_df, val_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df["label_encoded"], random_state=42)

# Datasets
train_dataset = SkinCancerDataset(train_df, image_dir, transform=unified_train_transforms)
val_dataset = SkinCancerDataset(val_df, image_dir, transform=unified_val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

Classes: ['ACK' 'BCC' 'MEL' 'NEV' 'SCC' 'SEK']


# **Models**

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(metadata_df["label_encoded"]),
    y=metadata_df["label_encoded"]
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

In [ ]:
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total


def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Validation", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

In [ ]:
# Class weights as a tensor
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10,
                scheduler=None, class_names=None, save_dir="./checkpoints", model_name="model"):
    os.makedirs(save_dir, exist_ok=True)
    model.to(device)

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    best_val_acc = 0.0
    best_model_path = os.path.join(save_dir, f"{model_name}_best.pth")

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        # Training loop
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", leave=False)
        for inputs, labels in train_loop:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            train_loop.set_postfix(loss=loss.item())

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        all_preds, all_labels = [], []

        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", leave=False)
        with torch.no_grad():
            for inputs, labels in val_loop:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= val_total
        val_acc = val_correct / val_total

        if scheduler:
            scheduler.step(val_loss)

        if val_acc > best_val_acc:
          best_val_acc = val_acc
          torch.save(model.state_dict(), best_model_path)
          print(f"Saved new best model at epoch {epoch+1} with accuracy {val_acc:.4f}")

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        # Print summary for the epoch
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    print(f"\nBest model saved to: {best_model_path}")

    return model, history, best_model_path

**DenseNet121**

In [ ]:
# Load DenseNet121 from timm
DenseNet121_model = timm.create_model("densenet121", pretrained=True, num_classes=num_classes)
DenseNet121_model.to(device)

# Optimiser
optimizer = optim.Adam(DenseNet121_model.parameters(), lr=3e-4)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

In [ ]:
# Train
trained_model, history, best_model_path = train_model(
    model=DenseNet121_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=10,
    class_names=class_names,
    save_dir="/content/drive/MyDrive/Final Year Project/Code/Image Classification/DenseNet121",
    model_name="pytorch_densenet121"
)


Epoch 1/10
Train Loss: 1.4293, Accuracy: 0.4266
Val   Loss: 1.1138, Accuracy: 0.4957



Epoch 2/10
Train Loss: 0.9703, Accuracy: 0.6317
Val   Loss: 1.0412, Accuracy: 0.5217



Epoch 3/10
Train Loss: 0.7621, Accuracy: 0.7035
Val   Loss: 0.9123, Accuracy: 0.5826



Epoch 4/10
Train Loss: 0.6258, Accuracy: 0.7372
Val   Loss: 0.9794, Accuracy: 0.5957



Epoch 5/10
Train Loss: 0.4636, Accuracy: 0.8058
Val   Loss: 0.9961, Accuracy: 0.6761



Epoch 6/10
Train Loss: 0.3899, Accuracy: 0.8243
Val   Loss: 1.0949, Accuracy: 0.6913



Epoch 7/10
Train Loss: 0.3718, Accuracy: 0.8531
Val   Loss: 1.0406, Accuracy: 0.6348



Epoch 8/10
Train Loss: 0.3486, Accuracy: 0.8634
Val   Loss: 0.9829, Accuracy: 0.6652



Epoch 9/10
Train Loss: 0.2677, Accuracy: 0.8896
Val   Loss: 1.0953, Accuracy: 0.7000



Epoch 10/10
Train Loss: 0.2284, Accuracy: 0.9162
Val   Loss: 1.6442, Accuracy: 0.6913

Best model saved to: /content/drive/MyDrive/Final Year Project/Code/Image Classification/DenseNet121/pytorch_densenet121_best.pth


**ResNet50V2**

In [ ]:
# Load ResNet50V2 from timm
ResNet50V2_model = timm.create_model("resnetv2_50x1_bitm", pretrained=True, num_classes=num_classes)

# Replace final head with your classifier if needed
ResNet50V2_model.reset_classifier(num_classes=num_classes)
ResNet50V2_model.to(device)

# Optimiser
optimizer = optim.Adam(ResNet50V2_model.parameters(), lr=3e-4)

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name resnetv2_50x1_bitm to current resnetv2_50x1_bit.goog_in21k_ft_in1k.
  model = create_fn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [ ]:
trained_model, history, best_model_path = train_model(
    model=ResNet50V2_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=10,
    class_names=class_names,
    save_dir="/content/drive/MyDrive/Final Year Project/Code/Image Classification/ResNet50V2",
    model_name="pytorch_resnet50v2"
)


Epoch 1/10
Train Loss: 1.8050, Accuracy: 0.2797
Val   Loss: 1.4615, Accuracy: 0.1848

Epoch 2/10
Train Loss: 1.4098, Accuracy: 0.3896
Val   Loss: 1.3477, Accuracy: 0.4739

Epoch 3/10
Train Loss: 1.3208, Accuracy: 0.4695
Val   Loss: 1.5090, Accuracy: 0.4565

Epoch 4/10
Train Loss: 1.1314, Accuracy: 0.5441
Val   Loss: 1.0853, Accuracy: 0.4478

Epoch 5/10
Train Loss: 1.0318, Accuracy: 0.5152
Val   Loss: 1.0685, Accuracy: 0.6043

Epoch 6/10
Train Loss: 1.1230, Accuracy: 0.5092
Val   Loss: 1.0929, Accuracy: 0.3457

Epoch 7/10
Train Loss: 0.9178, Accuracy: 0.5555
Val   Loss: 1.1017, Accuracy: 0.6022

Epoch 8/10
Train Loss: 0.8640, Accuracy: 0.6137
Val   Loss: 1.0575, Accuracy: 0.4304

Epoch 9/10
Train Loss: 0.8537, Accuracy: 0.6268
Val   Loss: 1.2076, Accuracy: 0.3413

Epoch 10/10
Train Loss: 0.8495, Accuracy: 0.6050
Val   Loss: 1.1116, Accuracy: 0.6457


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

**EfficientNetV2**

In [ ]:
# Load EfficientNetV2-S from timm
EffNetV2S_model = timm.create_model("tf_efficientnetv2_s", pretrained=True)
EffNetV2S_model.reset_classifier(num_classes=num_classes)
EffNetV2S_model.to(device)

# Optimiser
optimizer = optim.Adam(EffNetV2S_model.parameters(), lr=3e-4)

model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

In [ ]:
# Train
trained_model, history, best_model_path = train_model(
    model=EffNetV2S_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=10,
    class_names=class_names,
    save_dir="/content/drive/MyDrive/Final Year Project/Code/Image Classification/EfficientNetV2",
    model_name="pytorch_efficientnetv2s"
)

Saved new best model at epoch 1 with accuracy 0.5109

Epoch 1/10
Train Loss: 1.2048, Accuracy: 0.5131
Val   Loss: 1.0194, Accuracy: 0.5109


Saved new best model at epoch 2 with accuracy 0.5891

Epoch 2/10
Train Loss: 0.7272, Accuracy: 0.7111
Val   Loss: 0.8842, Accuracy: 0.5891


Saved new best model at epoch 3 with accuracy 0.6370

Epoch 3/10
Train Loss: 0.4075, Accuracy: 0.8281
Val   Loss: 0.9802, Accuracy: 0.6370


Saved new best model at epoch 4 with accuracy 0.7065

Epoch 4/10
Train Loss: 0.3625, Accuracy: 0.8466
Val   Loss: 0.9402, Accuracy: 0.7065



Epoch 5/10
Train Loss: 0.2589, Accuracy: 0.8857
Val   Loss: 1.1787, Accuracy: 0.6674


Saved new best model at epoch 6 with accuracy 0.7239

Epoch 6/10
Train Loss: 0.1949, Accuracy: 0.9211
Val   Loss: 1.3114, Accuracy: 0.7239



Epoch 7/10
Train Loss: 0.2050, Accuracy: 0.9124
Val   Loss: 1.0865, Accuracy: 0.7217


Saved new best model at epoch 8 with accuracy 0.7370

Epoch 8/10
Train Loss: 0.1283, Accuracy: 0.9543
Val   Loss: 1.3851, Accuracy: 0.7370



Epoch 9/10
Train Loss: 0.1207, Accuracy: 0.9559
Val   Loss: 1.2388, Accuracy: 0.7304



Epoch 10/10
Train Loss: 0.1168, Accuracy: 0.9516
Val   Loss: 1.2635, Accuracy: 0.6913

Best model saved to: /content/drive/MyDrive/Final Year Project/Code/Image Classification/EfficientNetV2/pytorch_efficientnetv2s_best.pth


**Swin Tiny**

In [ ]:
# Load Swin Tiny from timm
SwinTiny_model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=num_classes)
SwinTiny_model.to(device)

# Optimiser
optimizer = optim.AdamW(SwinTiny_model.parameters(), lr=3e-4, weight_decay=1e-4)

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [ ]:
# Train
trained_model, history, best_model_path = train_model(
    model=SwinTiny_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=10,
    class_names=class_names,
    save_dir="/content/drive/MyDrive/Final Year Project/Code/Image Classification/Swin-Tiny",
    model_name="pytorch_swintiny"
)

Saved new best model at epoch 1 with accuracy 0.3500

Epoch 1/10
Train Loss: 1.8334, Accuracy: 0.1850
Val   Loss: 1.7709, Accuracy: 0.3500


Saved new best model at epoch 2 with accuracy 0.3630

Epoch 2/10
Train Loss: 1.7910, Accuracy: 0.1665
Val   Loss: 1.7491, Accuracy: 0.3630



Epoch 3/10
Train Loss: 1.7097, Accuracy: 0.2231
Val   Loss: 1.6103, Accuracy: 0.2304


Saved new best model at epoch 4 with accuracy 0.3696

Epoch 4/10
Train Loss: 1.6508, Accuracy: 0.2987
Val   Loss: 1.4695, Accuracy: 0.3696



Epoch 5/10
Train Loss: 1.5968, Accuracy: 0.2845
Val   Loss: 1.8240, Accuracy: 0.0913



Epoch 6/10
Train Loss: 1.5865, Accuracy: 0.2709
Val   Loss: 1.5034, Accuracy: 0.3196


Saved new best model at epoch 7 with accuracy 0.4413

Epoch 7/10
Train Loss: 1.4586, Accuracy: 0.3303
Val   Loss: 1.3272, Accuracy: 0.4413



Epoch 8/10
Train Loss: 1.4692, Accuracy: 0.3890
Val   Loss: 1.4843, Accuracy: 0.3565


Saved new best model at epoch 9 with accuracy 0.5370

Epoch 9/10
Train Loss: 1.4307, Accuracy: 0.4157
Val   Loss: 1.2829, Accuracy: 0.5370



Epoch 10/10
Train Loss: 1.3749, Accuracy: 0.4091
Val   Loss: 1.3416, Accuracy: 0.4978

Best model saved to: /content/drive/MyDrive/Final Year Project/Code/Image Classification/Swin-Tiny/pytorch_swintiny_best.pth


**DeiT-Small**

In [ ]:
# Load DeiT Small from timm
DeiT_model = timm.create_model("deit_small_patch16_224", pretrained=True, num_classes=num_classes)
DeiT_model.to(device)

# Optimiser
optimizer = optim.AdamW(DeiT_model.parameters(), lr=3e-4, weight_decay=1e-4)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

In [ ]:
# Train
trained_model, history, best_model_path = train_model(
    model=DeiT_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=10,
    class_names=class_names,
    save_dir="/content/drive/MyDrive/Final Year Project/Code/Image Classification/DeiT-Small",
    model_name="pytorch_deitsmall"
)

Saved new best model at epoch 1 with accuracy 0.4239

Epoch 1/10
Train Loss: 1.7593, Accuracy: 0.2187
Val   Loss: 1.4767, Accuracy: 0.4239


Saved new best model at epoch 2 with accuracy 0.6043

Epoch 2/10
Train Loss: 1.3499, Accuracy: 0.4630
Val   Loss: 1.0742, Accuracy: 0.6043



Epoch 3/10
Train Loss: 1.1928, Accuracy: 0.5501
Val   Loss: 1.3057, Accuracy: 0.4326



Epoch 4/10
Train Loss: 1.0928, Accuracy: 0.5783
Val   Loss: 1.1530, Accuracy: 0.4043



Epoch 5/10
Train Loss: 0.9267, Accuracy: 0.6175
Val   Loss: 1.0439, Accuracy: 0.5848



Epoch 6/10
Train Loss: 0.8847, Accuracy: 0.6387
Val   Loss: 0.9921, Accuracy: 0.5630



Epoch 7/10
Train Loss: 0.7409, Accuracy: 0.6861
Val   Loss: 0.9668, Accuracy: 0.6043


Saved new best model at epoch 8 with accuracy 0.6152

Epoch 8/10
Train Loss: 0.6147, Accuracy: 0.7367
Val   Loss: 1.0737, Accuracy: 0.6152


Saved new best model at epoch 9 with accuracy 0.6500

Epoch 9/10
Train Loss: 0.7140, Accuracy: 0.7057
Val   Loss: 1.1395, Accuracy: 0.6500



Epoch 10/10
Train Loss: 0.6867, Accuracy: 0.7089
Val   Loss: 0.9559, Accuracy: 0.6022

Best model saved to: /content/drive/MyDrive/Final Year Project/Code/Image Classification/DeiT-Small/pytorch_deitsmall_best.pth
